In [3]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler, Imputer
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [2]:
! wget https://storage.googleapis.com/bdt-spark-store/external_sources.csv -O gcs_external_sources.csv

--2020-11-19 22:15:37--  https://storage.googleapis.com/bdt-spark-store/external_sources.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.170.16, 172.217.170.48, 172.217.170.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.170.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15503836 (15M) [text/csv]
Saving to: ‘gcs_external_sources.csv’

gcs_external_source 100%[===================>]  14.79M  9.83MB/s    in 1.5s    

2020-11-19 22:15:39 (9.83 MB/s) - ‘gcs_external_sources.csv’ saved [15503836/15503836]



In [3]:
! wget https://storage.googleapis.com/bdt-spark-store/internal_data.csv -O gcs_internal_data.csv

--2020-11-19 20:54:14--  https://storage.googleapis.com/bdt-spark-store/internal_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.170.48, 172.217.170.16, 172.217.170.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.170.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152978396 (146M) [text/csv]
Saving to: ‘gcs_internal_data.csv’

gcs_internal_data.c 100%[===================>] 145.89M  10.2MB/s    in 14s     

2020-11-19 20:54:29 (10.3 MB/s) - ‘gcs_internal_data.csv’ saved [152978396/152978396]



In [2]:
import os
memory = '8g'
pyspark_submit_args = ' --verbose \
 --driver-cores 1 \
 --driver-memory=16g \
 --executor-memory 2g \
 --num-executors 4 \
 --executor-cores 1 \
 --master local[4] \
 --deploy-mode client \
 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

sc = SparkSession.builder.appName('BDT_PBA3').getOrCreate()

In [4]:
internal_df = sc.read.csv('data/gcs_internal_data.csv', inferSchema=True, header=True)

In [5]:
external_df = sc.read.csv('data/gcs_external_sources.csv', inferSchema=True, header=True)

In [26]:
sc.conf.set("spark.sql.crossJoin.enabled", "true")
full_df = internal_df.join(external_df, internal_df.SK_ID_CURR==external_df.SK_ID_CURR)

In [27]:
full_df.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [85]:
full_df.select('TARGET').distinct().show()

+------+
|TARGET|
+------+
|     1|
|     0|
+------+



In [29]:
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                  'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                  'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                  'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                  'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']
df = full_df.select(columns_extract)

In [30]:
df.show(n=3)

+------------------+------------------+-------------------+----------+-------------+--------------------+---------------+-----------+-----------+-----------------+---------------+----------+-----------------+----------------------+----------------+----------------+-----------+------+
|      EXT_SOURCE_1|      EXT_SOURCE_2|       EXT_SOURCE_3|DAYS_BIRTH|DAYS_EMPLOYED| NAME_EDUCATION_TYPE|DAYS_ID_PUBLISH|CODE_GENDER|AMT_ANNUITY|DAYS_REGISTRATION|AMT_GOODS_PRICE|AMT_CREDIT|ORGANIZATION_TYPE|DAYS_LAST_PHONE_CHANGE|NAME_INCOME_TYPE|AMT_INCOME_TOTAL|OWN_CAR_AGE|TARGET|
+------------------+------------------+-------------------+----------+-------------+--------------------+---------------+-----------+-----------+-----------------+---------------+----------+-----------------+----------------------+----------------+----------------+-----------+------+
|0.6529430442014209|0.6984528683753916| 0.8004513396487078|    -15612|        -1106|Secondary / secon...|          -4018|          M|    38281.5|

In [42]:
numbers = ["EXT_SOURCE_1", 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE']

for col in numbers:
    df = df.withColumn(col, F.col(col).cast('double'))

In [43]:
train, test = df.randomSplit([0.8, 0.2],101)

In [75]:
print(train.groupBy('TARGET').count().select("TARGET","count", (F.col("count")/train.count()).alias("Prevelence")).show())
print(test.groupBy('TARGET').count().select("TARGET","count", (F.col("count")/test.count()).alias("Prevelence")).show())

+------+------+-------------------+
|TARGET| count|         Prevelence|
+------+------+-------------------+
|     1| 19890|0.08089640867124903|
|     0|225980|  0.919103591328751|
+------+------+-------------------+

None
+------+-----+-------------------+
|TARGET|count|         Prevelence|
+------+-----+-------------------+
|     1| 4935|0.08006034944274103|
|     0|56706|  0.919939650557259|
+------+-----+-------------------+

None


In [68]:
print('Training data shape: ', (train.count(), len(train.columns)))
print('Test data shape: ',(test.count(), len(test.columns)))

Training data shape:  (245870, 18)
Test data shape:  (61641, 18)


In [47]:
string_columns = ["CODE_GENDER", "NAME_EDUCATION_TYPE", "ORGANIZATION_TYPE", "NAME_INCOME_TYPE"]
numbers = ["EXT_SOURCE_1", 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE']

train = train.na.fill('missing', subset=string_columns)
test = test.na.fill('missing', subset=string_columns)

train = train.na.fill(999999999999999.99, subset=numbers)
test = test.na.fill(999999999999999.99, subset=numbers)

In [76]:
string_indexes = [StringIndexer(inputCol=c, outputCol='IDX_' + c, handleInvalid='keep',
                                    stringOrderType='alphabetAsc') for c in string_columns]
imputer = Imputer(strategy='median', missingValue=999999999999999.99,
                      inputCols=numbers, outputCols=['imp_' + n for n in numbers])
assembler = VectorAssembler(outputCol="features",inputCols=['imp_' + n for n in numbers] + ['IDX_' + c for c in string_columns])
rf = RandomForestClassifier(featuresCol="features", cacheNodeIds=True, labelCol='TARGET', maxBins=100, numTrees=100, impurity='gini').setSeed(50)

In [109]:
pipeline_stages = [string_indexes + [imputer, assembler, rf]]

In [125]:
pipeline = Pipeline(stages=pipeline_stages[0]).fit(train)

In [126]:
test_scored = pipeline.transform(test)

In [127]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='TARGET', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(test_scored)

In [128]:
print(accuracy)

0.919939650557259


In [129]:
import pandas as pd
featuresCol = "features"
dataset = test_scored
featureImp = pipeline.stages[-1].featureImportances

list_extract = []
for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
    list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
varlist = pd.DataFrame(list_extract)
varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
varlist

,idx,name,vals,score
0,0,imp_EXT_SOURCE_1,NaN,0.093981
1,1,imp_EXT_SOURCE_2,NaN,0.363942
2,2,imp_EXT_SOURCE_3,NaN,0.366477
3,3,imp_DAYS_BIRTH,NaN,0.041220
4,4,imp_DAYS_EMPLOYED,NaN,0.015472
5,5,imp_DAYS_ID_PUBLISH,NaN,0.004982
6,6,imp_AMT_ANNUITY,NaN,0.004996
7,7,imp_DAYS_REGISTRATION,NaN,0.001401
8,8,imp_AMT_GOODS_PRICE,NaN,0.013472
9,9,imp_AMT_CREDIT,NaN,0.003996


In [130]:
df = df.na.fill('missing', subset=string_columns)
df = df.na.fill(999999999999999.99, subset=numbers)
df = df.withColumn('TARGET', F.col('TARGET').cast('double'))
pipeline_stages = pipeline.stages
new_pipeline = Pipeline(stages=pipeline_stages)
best_params_model = new_pipeline.fit(df)
overfitted_ = best_params_model.transform(test)

In [131]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='TARGET', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(overfitted_)
print(accuracy)

0.919939650557259
